# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE
    # and then back ...
    supercell = ase_atoms(lattice, atoms, positions) * (repeat, 1, 1)
    lattice   = load_lattice(supercell)
    positions = load_positions(supercell)
    atoms = fill(Al, length(positions))

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(lattice, atoms, positions; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
ase_atoms(setup.model).write("al_supercell.png")

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
is_converged = DFTK.ScfConvergenceDensity(1e-4)  # Flag convergence based on density
self_consistent_field(aluminium_setup(1); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298381043318                   -0.85    5.0
  2   -8.300207033651       -2.74       -1.25    1.8
  3   -8.300447247005       -3.62       -1.88    3.0
  4   -8.300462175731       -4.83       -2.75    1.2
  5   -8.300464397436       -5.65       -3.08    4.1
  6   -8.300464499301       -6.99       -3.23    2.5
  7   -8.300464566741       -7.17       -3.36    1.2
  8   -8.300464599735       -7.48       -3.47    5.6
  9   -8.300464630105       -7.52       -3.67    1.1
 10   -8.300464638460       -8.08       -3.81    1.1
 11   -8.300464642972       -8.35       -4.05    1.9


In [4]:
self_consistent_field(aluminium_setup(2); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.67379887977                   -0.70    6.0
  2   -16.67793158808       -2.38       -1.14    1.0
  3   -16.67913525304       -2.92       -1.87    3.6
  4   -16.67924677015       -3.95       -2.65    4.2
  5   -16.67928516087       -4.42       -3.08    5.8
  6   -16.67928620995       -5.98       -3.48    1.9
  7   -16.67928621835       -8.08       -4.04    1.0


In [5]:
self_consistent_field(aluminium_setup(4); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32575591848                   -0.56    7.8
  2   -33.33427163626       -2.07       -1.00    1.5
  3   -33.33593079358       -2.78       -1.72    5.0
  4   -33.33613426885       -3.69       -2.64    6.6
  5   -33.33680159091       -3.18       -2.24    6.2
  6   -33.33683161308       -4.52       -2.30    1.4
  7   -33.33694377507       -3.95       -3.55    3.8
  8   -33.33694373350   +   -7.38       -3.64    4.8
  9   -33.33694379489       -7.21       -3.77    1.6
 10   -33.33694392650       -6.88       -4.93    3.6


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298628823317                   -0.85    5.0
  2   -8.300296325501       -2.78       -1.59    1.4
  3   -8.300462867813       -3.78       -2.71    4.6
  4   -8.300460387341   +   -5.61       -2.95    4.9
  5   -8.300464045628       -5.44       -3.37    1.5
  6   -8.300464592305       -6.26       -3.93    1.5
  7   -8.300464643633       -7.29       -5.01    2.8


In [7]:
self_consistent_field(aluminium_setup(4); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32606690182                   -0.56    7.1
  2   -33.33380601207       -2.11       -1.28    1.5
  3   -33.08342457523   +   -0.60       -1.29   10.0
  4   -30.63121352825   +    0.39       -0.85    4.4
  5   -33.16988732983        0.40       -1.31    4.6
  6   -33.33608222790       -0.78       -2.24    3.8
  7   -33.33013128221   +   -2.23       -2.11    3.4
  8   -33.30191988966   +   -1.55       -1.79    5.0
  9   -33.33655147047       -1.46       -2.61    4.2
 10   -33.33664394034       -4.03       -2.61    2.0
 11   -33.33679335247       -3.83       -2.82    1.5
 12   -33.33693245210       -3.86       -3.31    2.6
 13   -33.33694250184       -5.00       -3.77    3.6
 14   -33.33694383681       -5.87       -4.13    3.0


For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.